В этом ноутбуке я буду делать функцию, которая будет преобразовывать список, который мы получим после вызова функции client.get_klines() в формат, для входа в нейросеть

In [62]:
from binance.client import Client
from config import api_key,secret_key

asset = "BTCUSDT"
client = Client(api_key = api_key, api_secret = secret_key)
test1 = client.get_klines(symbol = asset, interval = "1m", limit = 15)
print(test1)

[[1612207680000, '33810.92000000', '33882.02000000', '33810.75000000', '33852.15000000', '40.04430900', 1612207739999, '1355479.70560130', 644, '22.34195000', '756149.15871463', '0'], [1612207740000, '33854.06000000', '33866.65000000', '33841.02000000', '33853.46000000', '15.50238500', 1612207799999, '524761.27036195', 505, '12.01687100', '406764.01313077', '0'], [1612207800000, '33851.94000000', '33886.25000000', '33845.57000000', '33870.43000000', '33.75955900', 1612207859999, '1143304.25559574', 653, '13.94824300', '472384.33071484', '0'], [1612207860000, '33870.42000000', '33964.00000000', '33868.69000000', '33921.58000000', '81.54963400', 1612207919999, '2767191.79988478', 1302, '55.10417600', '1869683.72373117', '0'], [1612207920000, '33921.57000000', '33942.94000000', '33906.41000000', '33923.97000000', '34.70966100', 1612207979999, '1177549.15099295', 521, '8.90116700', '301996.59713561', '0'], [1612207980000, '33923.96000000', '34049.62000000', '33918.04000000', '34040.6200000

Видим, что есть лишние столбцы, форматы строк, а не чисел, а также отсутствие столбцов месяца,дня и тд. Также, надо отмаштабировать

Соответственно, надо сделать следующее:

    1. Добавить столбцы месяца, дня, часа(+3), минуты
    
    2. Удалить столбцы, которых у нас нет в обучении
    
    3. Сделать тип данных float32
    
    4. Отмаштабировать, использую scaler, натренированный на обучающих данных

Сделаем все поэтапно для наглядности, а потом напишем резулльтирующую функцию

In [17]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import time



df = pd.DataFrame(data = test1)

In [18]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,1612203060000,33890.06000000,33890.83000000,33860.00000000,33876.89000000,27.80673100,1612203119999,941915.12362222,859,12.35184100,418415.12859611,0
1,1612203120000,33876.09000000,33890.00000000,33826.73000000,33858.98000000,45.83174700,1612203179999,1551698.43405506,956,17.44597700,590693.44792518,0
2,1612203180000,33858.98000000,33875.94000000,33834.06000000,33862.12000000,23.90619500,1612203239999,809302.87410830,684,12.53391700,424351.73552776,0
3,1612203240000,33862.12000000,33869.02000000,33826.02000000,33826.80000000,18.72895200,1612203299999,633886.72685005,712,10.40954500,352290.18216961,0
4,1612203300000,33826.79000000,33829.16000000,33780.01000000,33791.59000000,49.93586300,1612203359999,1688105.36262039,1173,13.46422200,455153.83644867,0


In [21]:
#добавим месяцы, часы, дни и минуты
df["month"] = pd.to_datetime(df[0], unit = "ms").dt.month
df["day"] = pd.to_datetime(df[0], unit = "ms").dt.day
df["hour"] = pd.to_datetime(df[0], unit = "ms").dt.hour + 3
df["minute"] = pd.to_datetime(df[0], unit = "ms").dt.minute

In [23]:
#удалям ненужные столбцы
df = df.drop(columns = [0,6,8,9,10,11])

In [26]:
df.head()

,1,2,3,4,5,7,month,day,hour,minute
0,33890.06000000,33890.83000000,33860.00000000,33876.89000000,27.80673100,941915.12362222,2,1,21,11
1,33876.09000000,33890.00000000,33826.73000000,33858.98000000,45.83174700,1551698.43405506,2,1,21,12
2,33858.98000000,33875.94000000,33834.06000000,33862.12000000,23.90619500,809302.87410830,2,1,21,13
3,33862.12000000,33869.02000000,33826.02000000,33826.80000000,18.72895200,633886.72685005,2,1,21,14
4,33826.79000000,33829.16000000,33780.01000000,33791.59000000,49.93586300,1688105.36262039,2,1,21,15


In [31]:
#преобразуем тип данных
df = df.astype("float32")

In [41]:
#обчаем scaler на обучающих данных
df_for_fit_scaler = pd.read_csv("df_for_fit_scaler.csv")
scaler = MinMaxScaler()
scaler.fit(df_for_fit_scaler)

In [46]:
scal = scaler.transform(df)

In [64]:
scal.shape

(15, 10)

вот, получили необходимый формат для входа в нейросеть

теперь можем объединить все и написать итоговую функцию

In [66]:
#функция будет принимать на вход список из 15 данных об последних минутах, а также обученный объект для маштабирования
def convert(array, scaler):
    df = pd.DataFrame(data = array)
    #добавляем данные из timestamp
    df["month"] = pd.to_datetime(df[0], unit = "ms").dt.month
    df["day"] = pd.to_datetime(df[0], unit = "ms").dt.day
    df["hour"] = pd.to_datetime(df[0], unit = "ms").dt.hour + 3
    df["minute"] = pd.to_datetime(df[0], unit = "ms").dt.minute
    #удаляем ненужные столбцы
    df = df.drop(columns = [0,6,8,9,10,11])
    #преобразуем в нужный тип данных
    df = df.astype("float32")
    #маштабируем
    result = scaler.transform(df)
    return result

In [67]:
#Посмотрим, сколько времени будет выполняться наша преобразование к нужному формату
start_time = time.time()
print(convert(test1,scaler))
print(time.time() - start_time)

[[1.74787676e+00 1.75164580e+00 1.74930286e+00 1.75005901e+00
  2.47789733e-02 1.29760802e-01 9.09090936e-02 1.73846881e-09
  9.56521749e-01 4.74576265e-01]
 [1.75015748e+00 1.75083315e+00 1.75090456e+00 1.75012839e+00
  9.59270354e-03 5.02356738e-02 9.09090936e-02 1.73846881e-09
  9.56521749e-01 4.91525412e-01]
 [1.75004554e+00 1.75186956e+00 1.75114536e+00 1.75102556e+00
  2.08900403e-02 1.09449126e-01 9.09090936e-02 1.73846881e-09
  9.56521749e-01 5.08474588e-01]
 [1.75102270e+00 1.75598037e+00 1.75236881e+00 1.75372982e+00
  5.04620112e-02 2.64904737e-01 9.09090936e-02 1.73846881e-09
  9.56521749e-01 5.25423706e-01]
 [1.75372696e+00 1.75486696e+00 1.75436461e+00 1.75385618e+00
  2.14779526e-02 1.12727411e-01 9.09090936e-02 1.73846881e-09
  9.56521749e-01 5.42372882e-01]
 [1.75385332e+00 1.76050746e+00 1.75497997e+00 1.76002383e+00
  5.48604652e-02 2.88568169e-01 9.09090936e-02 1.73846881e-09
  9.56521749e-01 5.59322059e-01]
 [1.76002038e+00 1.76055706e+00 1.75918663e+00 1.75880826e